In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
happiness_file = "../Data_Sources/world_happiness_2017.csv"
happiness_df = pd.read_csv(happiness_file)
#make the country lowecase
happiness_df = happiness_df.rename(columns = {'Country' : 'country'})
happiness_df.set_index('country',inplace = True)

alcohol_file = "../Data_Sources/HappinessAlcoholConsumption.csv"
alcohol_df = pd.read_csv(alcohol_file)
#make the country lowecase
alcohol_df = alcohol_df.rename(columns = {'Country' : 'country'})
alcohol_df.set_index('country',inplace = True)

In [3]:
happiness_df.head()

,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
country,,,,,,,,,,,
Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [4]:
alcohol_df.head()

,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
country,,,,,,,,
Denmark,Western Europe,north,7.526,928,53.579,224,81,278
Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
Iceland,Western Europe,north,7.501,933,60.530,233,61,78
Norway,Western Europe,north,7.498,951,70.890,169,71,129
Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [5]:
#for happiness file we will keep country, happiness score, Economy,  
happiness_df.columns

Index(['Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low',
       'Economy..GDP.per.Capita.', 'Family', 'Health..Life.Expectancy.',
       'Freedom', 'Generosity', 'Trust..Government.Corruption.',
       'Dystopia.Residual'],
      dtype='object')

In [6]:
#renaming some columns then remove some columns for the database
#im also making the columns lowercase so it will be easier for pgadmin4
happiness_df = happiness_df.rename(columns = {'Economy..GDP.per.Capita.':'gdp_percapita','Happiness.Score':'happines_score','Health..Life.Expectancy.':'health_life_expectancy','Trust..Government.Corruption.':'government_trust','Family':'family','Freedom':'freedom'})
#keeping the necessary columns
happiness_df = happiness_df[['happines_score','gdp_percapita','family', 'health_life_expectancy','freedom', 'government_trust']].copy()

In [7]:
#we will keep country, beer, spirit, and wine
alcohol_df.columns

Index(['Region', 'Hemisphere', 'HappinessScore', 'HDI', 'GDP_PerCapita',
       'Beer_PerCapita', 'Spirit_PerCapita', 'Wine_PerCapita'],
      dtype='object')

In [8]:
#making the necessary column lowercase for pgadmin4
alcohol_df = alcohol_df.rename(columns = {'Beer_PerCapita':'beer_per_capita','Spirit_PerCapita':'spirit_per_capita','Wine_PerCapita':'wine_per_capita'})

In [9]:
alcohol_df = alcohol_df[['beer_per_capita', 'spirit_per_capita','wine_per_capita']].copy()

In [10]:
#connecting to the server
rds_connection_string = "megan:megan@dataviz-saburo.chfkodcmbiws.us-west-1.rds.amazonaws.com:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
#adding creting and adding the tables to postgres
happiness_df.to_sql(name='world_happiness', con=engine, if_exists='replace', index=True)
alcohol_df.to_sql(name='world_alcohol', con=engine, if_exists='replace', index=True)